# Keras_Receptive_Fields
## 說明
簡中譯為感受野，繁中譯為接受區，國家教育研究院雙語詞彙中在魚類應用中為受精區，主要說明著在Convolution與Pooling之後的每一個元素所能包含到原始輸入圖像中範圍元素。  

舉個簡單的範例，輸入圖像為5x5，執行Convolution，filter_size=(3,3)，stride=1，在Convolution之後的output dimension為(3,3)，其output中的每一個元素皆包含上層(3x3)的資訊在裡面，再經過第二次的Convolution，filter_size(3,3)，stride=1，output dimension為(1,1)，這唯一的一個元素所包含的原始輸入圖像即(5x5)，意即receptive_fields為5。

如下圖所示：
![](./images/receptive_fields.jpg)

計算receptive_fields的用意在於，建置模型的同時我們可以理解轉fully-connected之前的output，每一個元素究竟能夠包含輸入圖像多少訊息在裡面，以Alexnet來說，它的pool5 layer的receptive_fields為195，而它的input shape為227，這數值是非常大的，以inception為例，它在保留極大的receptive_fields的情況下瘦身了模型，它的參數僅Alexnet的1/12，而VGG的參數量卻是Alexnet的3倍左右，但inception的效能卻不輸Alexnet與VGG。

因此這提供我們一個方向，模型建構的同時，如果能夠在保留極大receptive_fields的情況下減少參數量，對於訓練學習過程中的時間與建置完成的模型大小都有很大的幫助。
## 如何計算
參考如連結：http://shawnleezx.github.io/blog/2017/02/11/calculating-receptive-field-of-cnn/   

![](./images/receptive_fields_2.jpg)  
* $l_{k-1}$: 為k-1層的receptive_fields
* $f_k$: k層的filter_size<sub>假設filter_size的h,w是相同</sub>
* $s_i$: 為第i層的stride

## 範例
### 簡單範例
以上面說明範例計算
1. layer_1: 第一層的receptive field皆為1
2. layer_2: 1 + (3-1)\*1 = 1 + 2 = 3
3. layer_3: 3 + (3-1)\*1 = 3 + 2 = 5  

### 程式碼
#### 載入需求套件

In [1]:
# import numpy as np

# seed = 10
# np.random.seed(seed)

# import pandas as pd
# import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, AveragePooling2D, GlobalAveragePooling2D, Input, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.optimizers import SGD
# import os
# import tensorflow as tf
# from keras import backend as K

# %matplotlib inline

Using TensorFlow backend.


### 建構模型
模型依範例建構，沒有很正式，請不要太介意

In [15]:
model = Sequential()
model.add(Conv2D(1, (3,3), strides=(1,1), input_shape=(5,5,1)))
model.add(Conv2D(1, (3,3), strides=(1,1), input_shape=(5,5,1)))
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 3, 3, 1)           10        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 1, 1)           10        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


### 建立function
設置計算receptive fields的function，該function是在下在尋找資料的時候於網路上所得，但回頭尋找來源的時候遍尋不著並且網頁疑似已404，如作者發現在下是抄襲您的程式碼的時候請來信告知。

In [21]:
#  Receptive Fields(感受野)的計算
def print_receptive_fields(model):
    #  初始receptive fields為1
    r, s = 1, 1
    print('{:15}, {:4}, {:4}, {:4}, {:4}'.format('layer', ' r', ' s', ' R', ' S'))
    print('-' * 39)
    print('{:15}, {:4}, {:4}, {:4}, {:4}'.format('input', 1, 1, r, s))
    #  取得模型各layer資訊
    layer = model.input_layers[0]
    while layer is not None:
        if hasattr(layer, 'kernel_size'):
            #  假設filter_size與stride的h與w相同
            assert layer.kernel_size[0] == layer.kernel_size[1]
            assert layer.strides[0] == layer.strides[1]
            lr = layer.kernel_size[0]
            ls = layer.strides[0]
            r = r+(lr-1)*s
            s = s*ls
            print('{:15}, {:4}, {:4}, {:4}, {:4}'.format(layer.name, lr, ls, r, s))
        if hasattr(layer, 'pool_size'):
            #  假設filter_size與stride的h與w相同
            assert layer.pool_size[0] == layer.pool_size[1]
            assert layer.strides[0] == layer.strides[1]
            lr = layer.pool_size[0]
            ls = layer.strides[0]
            r = r+(lr-1)*s
            s = s*ls
            print('{:15}, {:4}, {:4}, {:4}, {:4}'.format(layer.name, lr, ls, r, s))
        if layer._outbound_nodes:
            layer = layer._outbound_nodes[0].outbound_layer
        else:
            layer = None
     

In [22]:
print_receptive_fields(model)      

layer          ,  r  ,  s  ,  R  ,  S  
---------------------------------------
input          ,    1,    1,    1,    1
conv2d_5       ,    3,    1,    3,    1
conv2d_6       ,    3,    1,    5,    1
